## Numba

In [1]:
from numba import cuda, float32
import numpy as np
import math
from fastcore.test import *

In [2]:
# sanity timer
def event_time(func, n_warm=20,n_time=20):
    for i in range(n_warm):
        func()
    cuda.synchronize()
    evtstart = cuda.event(timing=True)
    evtend = cuda.event(timing=True)
    evtstart.record()
    for i in range(n_time):
        func()
    evtend.record()
    evtend.synchronize()
    time_ms = cuda.event_elapsed_time(evtstart, evtend)/n_time
    print(f'Elapsed time using events: {time_ms:.2f}ms')
    return 

## CUDA - The importance of choosing an appropriate thread block size

Even when using a naive [unblocked](https://numba.pydata.org/numba-doc/dev/cuda/examples.html) matrix mult kernel it is important to choose an appropriate thread block size to avoid wasting performance.

There is a delicate interplay between thread block size and perfomance which is heavily dependant on the CUDA kernel.  This depends on the memory access pattern, the device architechture, shared memory usage etc.  To simplify matters we will just investigate the effect of two parameters which can make an impact when using a simple [kernel](https://numba.pydata.org/numba-doc/dev/cuda/examples.html) with no shared memory usage 
1. memory [coalescing](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#coalesced-access-to-global-memory) and,
2. utilized threads

while maintaingin full occupancy.

In [3]:
x_train_sz = (50000,784)
weights_sz= (x_train_sz[1],10)
x_train = np.random.rand(*x_train_sz);
weights = np.random.rand(*weights_sz);
r = np.zeros((x_train_sz[0],weights_sz[1]))
t = x_train@weights
m1g,m2g,rg = map(cuda.to_device,(x_train,weights,r))
m1g.shape,m2g.shape,rg.shape

((50000, 784), (784, 10), (50000, 10))

In [4]:
@cuda.jit
def matmul(a,b,c):
    i, j = cuda.grid(2)
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp

In [5]:
def calc_blocks(threads,dst_shape):
    blocks = (math.ceil(dst_shape[0] / threads[0]), math.ceil(dst_shape[1] / threads[1]))
    print(f'Grid - yBlocks: {blocks[0]}, xBlocks: {blocks[1]}, Threads per block - y: {threads[0]}, x: {threads[1]}')
    return blocks

Original thread block size 16x16.  
1. Writes will be uncoalesced (line size is 128 bytes, 32*sizeof(float), ideally thread block should have 32 x threads).
2. 6 = 16 - 10 threads will be wasted per row.

In [6]:
rg = cuda.to_device(r)
threads = (16,16)
blocks = calc_blocks(threads,rg.shape)
matmul[blocks,threads](m1g,m2g,rg)
test_close(t,rg.copy_to_host())

Grid - yBlocks: 3125, xBlocks: 1, Threads per block - y: 16, x: 16


In [10]:
%%timeit -n 10 
matmul[blocks,threads](m1g,m2g,rg)
cuda.synchronize()

5.73 ms ± 45.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
event_time(lambda: matmul[blocks,threads](m1g,m2g,rg))

Elapsed time using events: 5.53ms


What happens when we waste more threads per row, try block size 1x512.
1. Writes still uncoalesced (line size is 128 bytes, 32*sizeof(float), ideally thread block should have 32 x threads).
2. 502 = 512 - 10 threads will be wasted per row.

In [12]:
rg = cuda.to_device(r)
threads = (1,256)
blocks = calc_blocks(threads,rg.shape)
matmul[blocks,threads](m1g,m2g,rg)
test_close(t,rg.copy_to_host())

Grid - yBlocks: 50000, xBlocks: 1, Threads per block - y: 1, x: 256


In [15]:
%%timeit -n 10 
matmul[blocks,threads](m1g,m2g,rg)
cuda.synchronize()

15 ms ± 38.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
event_time(lambda: matmul[blocks,threads](m1g,m2g,rg))

Elapsed time using events: 14.61ms


What if we reduce cache hits by reading from new memory locations in m1g in every thread.
1. Writes still uncoalesced (line size is 128 bytes, 32*sizeof(float), ideally thread block should have 32 x threads)
2. 0 threads will be wasted per row

In [17]:
rg = cuda.to_device(r)
threads = (256,1)
blocks = calc_blocks(threads,rg.shape)
matmul[blocks,threads](m1g,m2g,rg)
test_close(t,rg.copy_to_host())

Grid - yBlocks: 196, xBlocks: 10, Threads per block - y: 256, x: 1


In [18]:
%%timeit -n 10 
matmul[blocks,threads](m1g,m2g,rg)
cuda.synchronize()

40.3 ms ± 60.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
event_time(lambda: matmul[blocks,threads](m1g,m2g,rg))

Elapsed time using events: 39.86ms


Transpose the matrices so we can use blocks of n*32 threads to coalesce the writes.

In [20]:
tT = weights.transpose()@x_train.transpose()
m1gT = m1g.transpose()
m2gT = m2g.transpose()
rgT = rg.transpose()
m1gT.shape,m2gT.shape,rgT.shape

/home/b/mambaforge/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 25 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


((784, 50000), (10, 784), (10, 50000))

In [45]:
rgT = cuda.to_device(rg.transpose())
threads = (1,256)
blocks = calc_blocks(threads,rgT.shape)
matmul[blocks,threads](m2gT,m1gT,rgT)
test_close(tT,rgT.copy_to_host())

Grid - yBlocks: 10, xBlocks: 196, Threads per block - y: 1, x: 256


In [49]:
%%timeit -n 10 
matmul[blocks,threads](m2gT,m1gT,rgT)
cuda.synchronize()

3.48 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
event_time(lambda: matmul[blocks,threads](m2gT,m1gT,rgT))

Elapsed time using events: 3.41ms
